In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import corr
import os
os.chdir('/home/ubuntu/data')

In [2]:
spark = SparkSession.builder.appName('Cust_churn').getOrCreate()

In [3]:
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True) 
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [4]:
for row in df.head(5):
    print(row, '\n')

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0) 

Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0) 

Row(A=6, B=2, C=13.0, D=6, Spoiled=1.0) 

Row(A=4, B=2, C=12.0, D=1, Spoiled=1.0) 

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0) 



In [5]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')
output = assembler.transform(df)
output.show()

+---+---+----+---+-------+-------------------+
|  A|  B|   C|  D|Spoiled|           features|
+---+---+----+---+-------+-------------------+
|  4|  2|12.0|  3|    1.0| [4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|    1.0| [5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|    1.0| [6.0,2.0,13.0,6.0]|
|  4|  2|12.0|  1|    1.0| [4.0,2.0,12.0,1.0]|
|  4|  2|12.0|  3|    1.0| [4.0,2.0,12.0,3.0]|
| 10|  3|13.0|  9|    1.0|[10.0,3.0,13.0,9.0]|
|  8|  5|14.0|  5|    1.0| [8.0,5.0,14.0,5.0]|
|  5|  8|12.0|  8|    1.0| [5.0,8.0,12.0,8.0]|
|  6|  5|12.0|  9|    1.0| [6.0,5.0,12.0,9.0]|
|  3|  3|12.0|  1|    1.0| [3.0,3.0,12.0,1.0]|
|  9|  8|11.0|  3|    1.0| [9.0,8.0,11.0,3.0]|
|  1| 10|12.0|  3|    1.0|[1.0,10.0,12.0,3.0]|
|  1|  5|13.0| 10|    1.0|[1.0,5.0,13.0,10.0]|
|  2| 10|12.0|  6|    1.0|[2.0,10.0,12.0,6.0]|
|  1| 10|11.0|  4|    1.0|[1.0,10.0,11.0,4.0]|
|  5|  3|12.0|  2|    1.0| [5.0,3.0,12.0,2.0]|
|  4|  9|11.0|  8|    1.0| [4.0,9.0,11.0,8.0]|
|  5|  1|11.0|  1|    1.0| [5.0,1.0,11.0,1.0]|
|  4|  9|12.0

In [6]:
final_data = output.select(['features', 'Spoiled'])
final_data.show()

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



In [7]:
rfc_model = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
rfc_model_fitted = rfc_model.fit(final_data)
rfc_model_fitted.featureImportances

SparseVector(4, {0: 0.0178, 1: 0.0112, 2: 0.9508, 3: 0.0202})

In [8]:
feature_imp_df = pd.DataFrame({'Feature':list(df.columns[:-1]), 'Importance':list(rfc_model_fitted.featureImportances)})
feature_imp_df.sort_values('Importance', ascending=False)

,Feature,Importance
2,C,0.950769
3,D,0.020172
0,A,0.017848
1,B,0.011212
